In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rvf10k/valid.csv
/kaggle/input/rvf10k/train.csv
/kaggle/input/rvf10k/rvf10k/valid/fake/5DIJ0KLS9U.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/WWRWBORX9A.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/MHMY7T7AHT.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/DF3VJDSVB0.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/MIM8TSAP37.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/L4JOZS03M8.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/L6RO5ZUF30.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/ZEEIEPN3VL.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/GFXAMQXKRZ.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/ZSRQ859YDA.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/LMBM5AZEAQ.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/FQIFB740AW.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/B8CA1K2O3F.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/9KS6OG0JCE.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/HVS4Q816MQ.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/AQ3V4LVXM8.jpg
/kaggle/input/rvf10k/rvf10k/valid/fake/FBY5X66SSK.jpg
/kaggle/input/rvf10k

In [6]:
train_df=pd.read_csv('/kaggle/input/rvf10k/train.csv')
valid_df=pd.read_csv('/kaggle/input/rvf10k/valid.csv')

In [3]:
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torchmetrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [4]:
class FacesDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.dataframe.iloc[idx, 5])
        image = Image.open(img_name)
        label = self.dataframe.iloc[idx, 3]

        if self.transform:
            image = self.transform(image)

        return image, label

In [7]:
# Define una transformación para convertir las imágenes a tensores para luego normalizarlas
# def calculus_transform(image) -> torch.Tensor:
#     transform = transforms.ToTensor()
#     return transform(image)

calculus_transform = transforms.ToTensor()


# Variables para calcular la media y la desviación estándar
mean = 0.
std = 0.
total_samples = 0

calculus_dataset = FacesDataset(dataframe=train_df, root_dir='/kaggle/input/rvf10k/rvf10k/', transform=calculus_transform)
# Crea un DataLoader para iterar sobre el conjunto de datos
calculus_loader = torch.utils.data.DataLoader(calculus_dataset, batch_size=1000, shuffle=True)


for images, _ in calculus_loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    total_samples += batch_samples

# Calcula la media y la desviación estándar finales
mean /= total_samples
std /= total_samples

print("Mean:", mean.numpy())
print("Std:", std.numpy())

Mean: [0.5212097  0.42598602 0.38105512]
Std: [0.24863924 0.22383371 0.221064  ]


In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean.numpy(), std.numpy()),
])

In [10]:
transform_aug = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(degrees=10),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.ColorJitter(saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean.numpy(), std.numpy())
])

In [11]:
# custom datasets
train_dataset = FacesDataset(dataframe=train_df, root_dir='/kaggle/input/rvf10k/rvf10k/', transform=transform)
valid_dataset = FacesDataset(dataframe=valid_df, root_dir='/kaggle/input/rvf10k/rvf10k/', transform=transform)

# data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [25]:
image_datasets = {}
image_datasets['train'] = train_dataset
image_datasets['val'] = valid_dataset
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [24]:
len(valid_dataset)

3000

- Adapto el código para los datal

In [21]:
dataloaders = {}
dataloaders['train'] = train_loader
dataloaders['val'] = valid_loader

In [12]:
# custom datasets
train_dataset_aug = FacesDataset(dataframe=train_df, root_dir='/kaggle/input/rvf10k/rvf10k/', transform=transform_aug)
valid_dataset_aug = FacesDataset(dataframe=valid_df, root_dir='/kaggle/input/rvf10k/rvf10k/', transform=transform)

# data loaders
train_loader_aug = DataLoader(train_dataset_aug, batch_size=32, shuffle=True)
valid_loader_aug = DataLoader(valid_dataset_aug, batch_size=32, shuffle=False)

In [13]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

## Fine Tunning

### Nuestra función de train

In [80]:
def train(model, optimizer, criterion, metric, data, epochs):

    train_loader = data["train"]
    valid_loader = data["valid"]

    if torch.cuda.is_available():
        model.to("cuda")
        metric.to("cuda")

    train_loss = []
    train_acc = []
    valid_loss = []
    valid_acc = []

    for epoch in range(epochs):

        # Pongo el modelo en modo entrenamiento
        model.train()

        epoch_train_loss = 0.0
        epoch_train_accuracy = 0.0

        for train_data, train_target in train_loader:

            if torch.cuda.is_available():
                train_data = train_data.to("cuda")
                train_target = train_target.to("cuda")
                #train_target = train_target.unsqueeze(2) # Se agrega por ValueError

            optimizer.zero_grad()
            output = model(train_data.float())
            loss = criterion(output, train_target.float())
            epoch_train_loss += loss.item()
            loss.backward()
            optimizer.step()

            accuracy = metric(output, train_target)
            epoch_train_accuracy += accuracy.item()

        epoch_train_loss = epoch_train_loss / len(train_loader)
        epoch_train_accuracy = epoch_train_accuracy / len(train_loader)
        train_loss.append(epoch_train_loss)
        train_acc.append(epoch_train_accuracy)

        # Pongo el modelo en modo testeo
        model.eval()

        epoch_valid_loss = 0.0
        epoch_valid_accuracy = 0.0

        for valid_data, valid_target in valid_loader:
            if torch.cuda.is_available():
                valid_data = valid_data.to("cuda")
                valid_target = valid_target.to("cuda")

            output = model(valid_data.float())
            epoch_valid_loss += criterion(output, valid_target.float()).item()
            epoch_valid_accuracy += metric(output, valid_target.float()).item()

        epoch_valid_loss = epoch_valid_loss / len(valid_loader)
        epoch_valid_accuracy = epoch_valid_accuracy / len(valid_loader)
        valid_loss.append(epoch_valid_loss)
        valid_acc.append(epoch_valid_accuracy)

        print("Epoch: {}/{} - Train loss {:.6f} - Train Accuracy {:.6f} - Valid Loss {:.6f} - Valid Accuracy {:.6f}".format(
        epoch+1, epochs, epoch_train_loss, epoch_train_accuracy, epoch_valid_loss, epoch_valid_accuracy))

    history = {}
    history["train_loss"] = train_loss
    history["train_acc"] = train_acc
    history["valid_loss"] = valid_loss
    history["valid_acc"] = valid_acc

    return history

In [33]:
def plot_history(history):

    fig, axs = plt.subplots(2, 1, figsize=(10, 10))
    axs[0].plot(history["train_loss"])
    axs[0].plot(history["valid_loss"])
    axs[0].title.set_text('Error de Entrenamiento vs Validación')
    axs[0].legend(['Train', 'Valid'])

    axs[1].plot(history["train_acc"])
    axs[1].plot(history["valid_acc"])
    axs[1].title.set_text('Accuracy de Entrenamiento vs Validación')
    axs[1].legend(['Train', 'Valid'])

In [35]:
def evaluate_model(model, valid_loader, metric):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    seed = 42
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            #_, predicted = torch.max(outputs, 1)
            predicted = outputs
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # print(f'Accuracy: {accuracy_score(all_labels, all_preds)}')
    # metric_result = metric(all_preds, all_labels)
    metric_result = metric(torch.tensor(all_preds), torch.tensor(all_labels))
    print(f'Accuracy: {metric_result}')

    # report
    all_preds_binary = [1 if value > 0.5 else 0 for value in all_preds]

    print("Classification Report:")
    print(classification_report(all_labels, all_preds_binary))

    # confusion matrix
    cm = confusion_matrix(all_labels, all_preds_binary)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='g')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [37]:
# Definimos una función que plotee los resultados
def plot_predictions(model,
                     valid_loader,
                     label_map = {0: 'fake', 1: 'real'},
                     device = device):
    plt.figure(figsize=(10, 10))

    images, labels = next(iter(valid_loader))

    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    # _, predicted = torch.max(outputs, 1)
    predicted = outputs
    predicted_binary = [1 if value > 0.5 else 0 for value in predicted]

    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].cpu().permute(1, 2, 0))
        color = 'green' if predicted_binary[i] == labels[i] else 'red'
        plt.title(f'Actual: {label_map[int(labels[i])]}, Predicted: {label_map[predicted_binary[i]]}', color=color)
        plt.axis('off')

    plt.show()

In [39]:
# Bajamos modelo pre-entrenado ResNet-18
model_resnet = models.resnet18(pretrained=True)

In [40]:
# Cortamos la última capa
num_ftrs = model_resnet.fc.in_features
num_ftrs

512

In [72]:
# Definimos correctamente la arquitectura
model_resnet.fc = nn.Linear(num_ftrs, 1)
model_resnet
#model_resnet.fc.shape

#model_resnet = model_resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [93]:
class ResNet18_Binary(nn.Module):

    def __init__(self, num_classes=2):
        super().__init__()

        # Pre-trained ResNet18 model
        self.resnet = models.resnet18(pretrained=True)

        # Replace the final fully-connected layer with a binary classifier
        self.fc = nn.Linear(in_features=512, out_features=num_classes)
        self.final = torch.nn.Sigmoid()

    def forward(self, x):
        # Pass the input through the ResNet18 model
        features = self.resnet(x)

        # Determine the flattened size dynamically
        flattened_size = features.numel()  # Number of elements in the tensor

        # Flatten the features using the correct size
        features = features.view(-1, flattened_size)

        # Apply the binary classifier
        output = self.fc(features)
        output = self.final(output)

        return output.squeeze()

In [94]:
# cantidad_clases = 2
images_width = 224
images_height = 224

# Example usage
model_resnet2 = ResNet18_Binary(num_classes=2)

optimizer = torch.optim.Adam(model_resnet2.parameters(), lr=0.0001)
# loss = torch.nn.CrossEntropyLoss()
loss = torch.nn.BCELoss()
# metric = torchmetrics.Accuracy(task='multiclass', num_classes=cantidad_clases)
metric = torchmetrics.classification.BinaryAccuracy()
data = {"train": train_loader, "valid": valid_loader, "image_width": images_width, "image_height": images_height}

history = train(model_resnet2,
                optimizer,
                loss,
                metric,
                data,
                10)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x32000 and 512x2)

### Función train de AP

In [17]:
"""def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model """

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
model_ft = models.resnet18(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 127MB/s] 


In [15]:
num_ftrs = model_ft.fc.in_features
num_ftrs

512

In [19]:
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

## Cambiando solo la FC a una ResNet-18

In [26]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=7)

Epoch 0/6
----------
train Loss: 0.1722 Acc: 0.9336
val Loss: 0.1933 Acc: 0.9217

Epoch 1/6
----------
train Loss: 0.0618 Acc: 0.9793
val Loss: 0.1561 Acc: 0.9393

Epoch 2/6
----------
train Loss: 0.0269 Acc: 0.9924
val Loss: 0.1495 Acc: 0.9440

Epoch 3/6
----------
train Loss: 0.0131 Acc: 0.9969
val Loss: 0.1317 Acc: 0.9503

Epoch 4/6
----------
train Loss: 0.0085 Acc: 0.9983
val Loss: 0.1255 Acc: 0.9550

Epoch 5/6
----------
train Loss: 0.0087 Acc: 0.9976
val Loss: 0.1599 Acc: 0.9460

Epoch 6/6
----------
train Loss: 0.0076 Acc: 0.9981
val Loss: 0.1484 Acc: 0.9500

Training complete in 6m 28s
Best val Acc: 0.955000


## Analizamos con una GoogLeNet

In [27]:
model_gn = models.googlenet(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 163MB/s]


In [28]:
num_ftrs = model_gn.fc.in_features
num_ftrs

1024

In [30]:
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_gn.fc = nn.Linear(num_ftrs, 2)

model_gn = model_gn.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_gn = optim.SGD(model_gn.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_gn, step_size=7, gamma=0.1)

In [31]:
model_gn = train_model(model_gn, criterion, optimizer_gn, exp_lr_scheduler,
                       num_epochs=7)

Epoch 0/6
----------
train Loss: 0.5349 Acc: 0.7236
val Loss: 0.4174 Acc: 0.8097

Epoch 1/6
----------
train Loss: 0.3134 Acc: 0.8670
val Loss: 0.2926 Acc: 0.8753

Epoch 2/6
----------
train Loss: 0.1783 Acc: 0.9301
val Loss: 0.2354 Acc: 0.9023

Epoch 3/6
----------
train Loss: 0.0896 Acc: 0.9704
val Loss: 0.1929 Acc: 0.9233

Epoch 4/6
----------
train Loss: 0.0474 Acc: 0.9870
val Loss: 0.2186 Acc: 0.9213

Epoch 5/6
----------
train Loss: 0.0300 Acc: 0.9926
val Loss: 0.2017 Acc: 0.9263

Epoch 6/6
----------
train Loss: 0.0209 Acc: 0.9960
val Loss: 0.1851 Acc: 0.9343

Training complete in 6m 54s
Best val Acc: 0.934333
